In [1]:
# %pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import scipy.integrate
import nugridpy.utils as utils
import sys
import os
import time

# if you make changes to the ppmpy module (e.g. add your analysis methods via a pull 
# request) in the https://github.com/PPMstar/PyPPM repo you may want use that 
# updated version
#sys.path.insert(0,'/user/david/PyPPM/')
sys.path.insert(0,'/home/user/home/PyPPM/')
from ppmpy import ppm

cb = utils.linestylecb # colours

#### Loading In Data

| index | quantity            |
|:-----:|:--------------:     |
| 0     | x                   |
| 1     | $\vec{u_{x}}$                  |   
| 2     | $\vec{u_{y}}$             | 
|  3    | $\vec{u_{z}}$             | 
|  4    | $\lvert\vec{u_{t}}\rvert$         | 
|  5    | $\lvert\vec{u_{r}}\rvert$             | 
|  6    | $\lvert\vec{\omega}\rvert$      | 
|  7    | P              | 
|  8    | rho            | 
|  9    | fv             | 

* Note that these are just 10 out of 32 quantities that can be made available in the moms data. 
* fv is the fractional volume of the material initially only outside the convection zone.

In [2]:
dir_repo = '/data/ASDR'
dir_project = 'H-core-M25'
rprof = {}; moms = {}         # initialize dictionaries to hold rprof and moms instances

# hold moms dumps for highres and low res
moms_dumps = []

runs = ['M29-768']            # select runs
moms_dumps.append(650)        # select dump numbers for moms

add_highres = False
if add_highres:
    runs.append('M35-1536')
    moms_dumps.append(375)  

# We can call get with a string now!
varlocs = ['xc','ux','uy','uz','|ur|','|ut|','|w|','P','rho','fv']

# We can now hold multiple dumps in memory
dumps_in_mem = 2

# rprof instance holds radial profiles for all dumps
# moms instance holds two dumps at a time in this case
for i,runid in enumerate(runs):        
    path = os.path.join(dir_repo,dir_project,runid)
    # radial profile:
    rprof[runid] = ppm.RprofSet(os.path.join(path,'rprofs'))
    moms[runid] = ppm.MomsDataSet(os.path.join(path,'myavsbq'),init_dump_read=moms_dumps[i],
                                dumps_in_mem=dumps_in_mem,var_list=varlocs)
print("moms and rprof dictionary created")

# We will work with 768 data for the rest of the notebook
runid = runs[0]

2413 rprof files found in '/data/ASDR/H-core-M25/M29-768/rprofs/.
Dump numbers range from 0 to 2412.
Reading history file '/data/ASDR/H-core-M25/M29-768/rprofs/HcoreE00768-0000.hstry'.
2661 .aaa files found in '/data/ASDR/H-core-M25/M29-768/myavsbq/.
Dump numbers range from 0 to 2412.
moms and rprof dictionary created


#### New Handling of Data

All variables obtained through get methods to the user are COPIES of the actual data. The user never gets the stored data (this ensures we can remove the data from memory when reading new dumps).

In [3]:
# lets work with just the 768 for now
thisrprof = rprof[runid]
thismoms = moms[runid]

# test a couple new features, get cartesian grid
x,y,z = thismoms.get_cgrid()

# we can now get the grid in spherical coordinates instead
r,theta,phi = thismoms.get_sgrid()

# let's get a z-plane, index notations: x[z,y,x]
print('The average and std of z values on a z-plane can be easily obtained with z[0,:,:]')
print(np.mean(z[0,:,:]),np.std(z[0,:,:]))
print('This is the "bottom" z-plane of the simulation')

The average and std of z values on a z-plane can be easily obtained with z[0,:,:]
-2486.9789237976074 0.0
This is the "bottom" z-plane of the simulation


## Grid Interpolation

We can now make plots of any quantity at the surface of a sphere. This is possible with a trillinear or "moments" (quadratic) interpolation of a quantity on the grid

In [4]:
# let's take a look at the radial velocities at the convective boundary
# we will say that the convective boundary is when the gradient of FV is largest
conv_radius = thisrprof.bound_rad([thismoms.what_dump_am_i],500.,1800.,'FV','max_grad')

# problem is in the data we have |ur|, what about direction?
# we can use ux, uy and uz to construct ur, utheta and uphi!
# looking at the docs, this function gives me a list which contains ur, utheta and uphi
u = thismoms.get_spherical_components('ux','uy','uz')
ur = u[0]
utheta = u[1]
uphi = u[2]

# the norm of |u| should be the same whether it is in cartesian or spherical, check
# we can get a norm with variables within data by calling their strings we assigned them earlier
# as well as supplying our own arrays to the method for derived quantities
u_normc = thismoms.norm('ux','uy','uz')
u_norms = thismoms.norm(ur,utheta,uphi)

print('The percent difference between the mean of these norms is {:0.2e}'.
      format(np.mean((u_normc - u_norms)/u_normc)))

The percent difference between the mean of these norms is 1.01e-08


With all rprof and momsdata methods there are docstrings to help the user. They should be detailed enough for you to understand what is going on like below:

In [5]:
thisrprof.bound_rad?

Signature: thisrprof.bound_rad(cycles, r_min, r_max, var='ut', criterion='min_grad', var_value=None, return_var_scale_height=False, eps=1e-09)
Docstring:
Method that finds the radius of a convective boundary.

If the search is based on gradients, second-order centred finite
differences are used to compute the gradient. The radius where
the gradient reaches a local extreme (minimum or maximum depending
on the value of criterion) is found. This radius is further
refined by fitting a parabola to the three points around the
local extreme and computing the radius at that the parabola
reaches its extreme.

If a certain value of var is searched for the method finds two
cells between which var crosses the requested value and then it
computes the radius of the actual crossing by linear interpolation.

Parameters
----------
cycles : list
    Cycle numbers to be used in the analysis.
r_min : float
    Minimum radius to for the boundary search.
r_max : float
    Maximum radius to for the boundary 

In [6]:
thismoms.get_spherical_interpolation?

Signature: thismoms.get_spherical_interpolation(varloc, radius, fname=None, method='trilinear', derivative='', logvarloc=False, coefficients=False, get_igrid=False, plot_mollweide=True, npoints=5000)
Docstring:
Returns the interpolated array of values of 'varloc' at a radius of 'radius' for a computed uniform distribution of points,
'npoints', on that sphere(s). It can return the 'theta,phi' (mollweide) coordinates of the 'varloc' values as well.

Parameters
----------
varloc: str, int, np.ndarray
    String: for the variable you want if defined on instantiation
    Int: index location of the variable you want
    np.ndarray: quantity you want to have interpolated on the grid
radius: float or np.ndarray
    The radius of the sphere you want 'varloc' to be interpolated to
fname: None,int
    None: default option, will grab current dump
    int: Dump number
method: str
    'trilinear' (fast): Use a trilinear method to interpolate onto the points on igrid
    'moments' (slower): Use a mom

In [47]:
# so our ur is fine, get its trillinear interpolated value at the convective boundary
# scale velocity by it's norm
ur_at_r, theta_grid, phi_grid = thismoms.get_spherical_interpolation(ur/u_normc,conv_radius)
utheta_at_r = thismoms.get_spherical_interpolation(utheta/u_normc,conv_radius,plot_mollweide=False)
uphi_at_r = thismoms.get_spherical_interpolation(uphi/u_normc,conv_radius,plot_mollweide=False)

# get the density on the boundary
p_at_r = thismoms.get_spherical_interpolation('P',conv_radius,plot_mollweide=False,method='moments')

# to understand where you are looking on a mollweide plot, plot xc
xc = thismoms.get_spherical_interpolation('xc',conv_radius,plot_mollweide=False)

# figure counter
# ifig = 0

In [8]:
# now we can plot with a mollweide projection
plt.close(ifig); ifig +=1; plt.figure(ifig)

# we want a mollweide projection
ax = plt.subplot(111, projection = 'mollweide')

# create the triangles that define boundaries for the values
triang = tri.Triangulation(phi_grid,theta_grid)
plt.tripcolor(triang,xc,cmap='RdBu_r')
cbar = plt.colorbar()
cbar.set_label('xc')

ax.set_xticklabels([])
ax.set_yticklabels([])

plt.title('R = {:0.0f} Mm'.format(conv_radius[0]))

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee420fd860>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee420fd860>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0.5, 1.0, 'R = 1534 Mm')

In [9]:
# now we can plot with a mollweide projection
plt.close(ifig); ifig +=1; plt.figure(ifig)

# we want a mollweide projection
ax = plt.subplot(111, projection = 'mollweide')

# create the triangles that define boundaries for the values
triang = tri.Triangulation(phi_grid,theta_grid)

# scale velocity by it's norm
plt.tripcolor(triang,ur_at_r,cmap='RdBu_r')
cbar = plt.colorbar()
cbar.set_label('ur / |u|')

ax.set_xticklabels([])
ax.set_yticklabels([])

plt.title('R = {:0.0f} Mm'.format(conv_radius[0]))

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee40e05208>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee40e05208>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0.5, 1.0, 'R = 1534 Mm')

In [49]:
# now we can plot with a mollweide projection
plt.close(ifig); ifig +=1; plt.figure(ifig)

# we want a mollweide projection
ax = plt.subplot(111, projection = 'mollweide')

# create the triangles that define boundaries for the values
triang = tri.Triangulation(phi_grid,theta_grid)

# scale velocity by it's norm
plt.tripcolor(triang,p_at_r,cmap='RdBu_r')
cbar = plt.colorbar()
cbar.set_label(r'P')

ax.set_xticklabels([])
ax.set_yticklabels([])

plt.title('R = {:0.0f} Mm'.format(conv_radius[0]))

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fedf5a802e8>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fedf5a802e8>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0.5, 1.0, 'R = 1534 Mm')

The flow is mainly driven by a dipole with flows moving right through the center. This creates positive and negative ur at different parts of the sphere we are looking at

In [13]:
d_radius = 100.
ur_dipole = thismoms.get_spherical_interpolation(ur/u_normc,d_radius,plot_mollweide=False)

In [14]:
# now we can plot with a mollweide projection
ifig +=1; plt.figure(ifig)

# we want a mollweide projection
ax = plt.subplot(111, projection = 'mollweide')

# create the triangles that define boundaries for the values
triang = tri.Triangulation(phi_grid,theta_grid)

# scale velocity by it's norm
plt.tripcolor(triang,ur_dipole,cmap='RdBu_r')
cbar = plt.colorbar()
cbar.set_label('ur / |u|')

ax.set_xticklabels([])
ax.set_yticklabels([])

plt.title('R = {:0.0f} Mm'.format(d_radius))

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee0397eb38>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fee0397eb38>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0.5, 1.0, 'R = 100 Mm')